In [ ]:
!pip install --upgrade interpret --quiet

     |████████████████████████████████| 6.6 MB 10.1 MB/s 
     |████████████████████████████████| 756 kB 54.0 MB/s 
     |████████████████████████████████| 280 kB 37.5 MB/s 
     |████████████████████████████████| 9.6 MB 42.2 MB/s 
     |████████████████████████████████| 3.6 MB 46.3 MB/s 
     |████████████████████████████████| 5.8 MB 26.2 MB/s 
     |████████████████████████████████| 564 kB 51.7 MB/s 
     |████████████████████████████████| 793 kB 41.5 MB/s 
     |████████████████████████████████| 130 kB 57.7 MB/s 
     |████████████████████████████████| 275 kB 64.7 MB/s 
     |████████████████████████████████| 251 kB 56.6 MB/s 
     |████████████████████████████████| 380 kB 38.6 MB/s 
     |████████████████████████████████| 81 kB 8.9 MB/s 
     |████████████████████████████████| 357 kB 50.7 MB/s 
     |████████████████████████████████| 65 kB 2.6 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the sourc

In [ ]:
import numpy as np

from sklearn.datasets import fetch_california_housing, load_breast_cancer, load_iris
from sklearn.model_selection import train_test_split

from interpret.glassbox import (
    ExplainableBoostingRegressor,
    ExplainableBoostingClassifier,
    RegressionTree,
    ClassificationTree
    )

In [ ]:
X, y = fetch_california_housing(return_X_y=True, as_frame=True)

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=.8, test_size=.2, random_state=42)

In [ ]:
ebm = ExplainableBoostingRegressor()

ebm.fit(
    X=X_train,
    y=y_train
)

ExplainableBoostingRegressor(feature_names=['MedInc', 'HouseAge', 'AveRooms',
                                            'AveBedrms', 'Population',
                                            'AveOccup', 'Latitude', 'Longitude',
                                            'HouseAge x AveOccup',
                                            'MedInc x AveOccup',
                                            'Latitude x Longitude',
                                            'MedInc x HouseAge',
                                            'HouseAge x Longitude',
                                            'AveOccup x Latitude',
                                            'Population x AveOccup',
                                            'MedInc x Longitude',
                                            'AveOccup x Longitude',
                                            'AveBedrms x Longitude'],
                             feature_types=['continuous', 'continuous',
                         

In [ ]:
ebm.score(
    X=X_valid,
    y=y_valid
)

0.8094346473894742

In [ ]:
from interpret import show

ebm_global = ebm.explain_global()
show(ebm_global)

In [ ]:
ebm_local = ebm.explain_local(X_valid, y_valid)
show(ebm_local)

In [ ]:
# 代理モデル

X_pertubated = X_train.copy()

for col in X_train.columns:
    X_pertubated[col] = X_pertubated[col].sample(frac=1)

y_pertubated = ebm.predict(X_pertubated)

In [ ]:
explainer = ClassificationTree()

explainer.fit(
    X=X_pertubated,
    y=y_pertubated
)

In [ ]:
explainer.score(
    X=X_pertubated,
    y=y_pertubated
)

0.978021978021978

In [ ]:
explainer_global = explainer.explain_global()

show(explainer_global)

In [ ]:
explainer_local_train = explainer.explain_local(
    X=X_train,
    y=ebm.predict(X_train)
)

show(explainer_local_train)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
explainer_local_valid = explainer.explain_local(
    X=X_valid,
    y=ebm.predict(X_valid)
)

show(explainer_local_valid)

In [ ]:
# find weekness

for col in X_valid.columns:
    explainer = RegressionTree(
        max_depth=3
        )

    explainer.fit(
        X=X_valid[[col]],
        y=np.power(y_valid - ebm.predict(X_valid), 2) * 10
    )

    explainer_global = explainer.explain_global()
    show(explainer_global)

In [ ]:
from itertools import permutations

for col1, col2 in permutations(X_train.columns, 2):
    explainer = RegressionTree(
        max_depth=3
        )

    explainer.fit(
        X=X_valid[[col1, col2]],
        y=np.power(y_valid - ebm.predict(X_valid), 2) * 10
    )

    explainer_global = explainer.explain_global()
    show(explainer_global)